In [4]:
# ==============================
# 1. IMPORT REQUIRED LIBRARIES
# ==============================

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

# ==============================
# 2. IMAGE PREPROCESSING
# ==============================

# This rescales image pixels from 0-255 to 0-1 (helps training)
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20, 
    width_shift_range=0.2,      # Horizontal shift
    height_shift_range=0.2,     # Vertical shift
    shear_range=0.2,            # Shear transformation
    zoom_range=0.4,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill new pixels after transformation
    )
test_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,          # Random rotation ±20 degrees
    width_shift_range=0.2,      # Horizontal shift
    height_shift_range=0.2,     # Vertical shift
    shear_range=0.2,            # Shear transformation
    zoom_range=0.4,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode='nearest'         # Fill new pixels after transformation
    )

# Load training images
train_data = train_datagen.flow_from_directory(
    r"D:\DATA SCIENCE\DL\PROJECTS\car_bike\Car-Bike-Dataset\Training_Set",      # path to training folder
    target_size=(128,128), # resize all images to 64x64
    batch_size=32,
    class_mode='binary'   # car vs bike (2 classes)
)

# Load testing images
test_data = test_datagen.flow_from_directory(
    r"D:\DATA SCIENCE\DL\PROJECTS\car_bike\Car-Bike-Dataset\Test_Set",
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)

# ==============================
# 3. BUILD CNN MODEL
# ==============================

model = Sequential()

# ---- Convolution Layer 1 ----
# Learns edges and simple shapes
model.add(Conv2D(
    filters=32,
    kernel_size=(3,3),
    activation='relu',
    input_shape=(128,128,3)
))

# Conv Layer 2 (NEW)
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# Conv Layer 3 (NEW)
model.add(Conv2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# ---- Convolution Layer 2 ----
# Learns more complex features
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())

# ---- Flatten ----
# Converts 2D feature maps into 1D vector
model.add(Flatten())

# ---- Fully Connected Layer ----
model.add(Dense(128, activation='relu'))

# ---- Output Layer ----
# 1 neuron because binary classification
model.add(Dense(1, activation='sigmoid'))

# ==============================
# 4. COMPILE THE MODEL
# ==============================

model.compile(
    optimizer='adam',              # controls learning speed
    loss='binary_crossentropy',    # for 2 classes
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)





Found 4000 images belonging to 2 classes.
Found 798 images belonging to 2 classes.


In [5]:
# 5. TRAIN THE MODEL
# ==============================

model.fit(
    train_data,
    epochs=30,          # increase for better accuracy
    validation_data=test_data
)


Epoch 1/30
 17/125 ━━━━━━━━━━━━━━━━━━━━ 3:00 2s/step - accuracy: 0.5724 - loss: 1.5220 - precision: 0.6050 - recall: 0.5567

C:\Users\HP ELITE BOOK\AppData\Local\Programs\Python\Python313\Lib\site-packages\PIL\Image.py:1039: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


125/125 ━━━━━━━━━━━━━━━━━━━━ 258s 2s/step - accuracy: 0.6973 - loss: 0.9682 - precision: 0.7030 - recall: 0.6830 - val_accuracy: 0.4987 - val_loss: 1.1662 - val_precision: 0.4987 - val_recall: 1.0000
Epoch 2/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 215s 2s/step - accuracy: 0.7945 - loss: 0.5455 - precision: 0.8011 - recall: 0.7835 - val_accuracy: 0.5013 - val_loss: 3.5190 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.8418 - loss: 0.4122 - precision: 0.8379 - recall: 0.8475 - val_accuracy: 0.5113 - val_loss: 0.8843 - val_precision: 0.5112 - val_recall: 0.4598
Epoch 4/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 183s 1s/step - accuracy: 0.8692 - loss: 0.3158 - precision: 0.8672 - recall: 0.8720 - val_accuracy: 0.6930 - val_loss: 0.5977 - val_precision: 0.6349 - val_recall: 0.9045
Epoch 5/30
125/125 ━━━━━━━━━━━━━━━━━━━━ 174s 1s/step - accuracy: 0.8790 - loss: 0.3005 - precision: 0.8782 - recall: 0.8800 - val_accuracy: 0.6704 - val_loss: 

In [6]:
# ==============================
# 6. SAVE THE MODEL
# ==============================

model.save("car_bike_cnn_model.h5")

print("Model training complete and saved!")

Model training complete and saved!


In [11]:
from tensorflow.keras.preprocessing import image

img = image.load_img(r"C:\Users\HP ELITE BOOK\Downloads\pexels-avinashpatel-445399.jpg", target_size=(128,128))
img = image.img_to_array(img)
img = img / 255.0
img = np.expand_dims(img, axis=0)

prediction = model.predict(img)

pred = model.predict(img)[0][0]

if pred > 0.80:
    print("Car")
elif pred < 0.20:
    print("Bike")
else:
    print("Unknown Image")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Bike


In [ ]:
# Evaluate the model on test data
results = model.evaluate(test_data)

# Print metric names and their values
for name, value in zip(model.metrics_names, results):
    print(f"{name}: {value:.4f}")
